In [9]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import Workspace, AmlCompute, Datastore
from azure.core.exceptions import ResourceNotFoundError
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes
import azure.ai.ml._artifacts._artifact_utilities as artifact_utils
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
import pandas as pd
import datetime
import os
from pathlib import Path
from ultralytics import YOLO

In [2]:
subscription_id = '31bc5bb0-1143-4d0c-ba02-570ecc764364'
resource_group = 'Face_Attribute_Recognition'
workspace_name = 'test_ws'
cluster_name = 'cpu-cluster'

In [3]:
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace_name)
ws = ml_client._workspaces.get(name = workspace_name)

In [4]:
try:
    cpu_cluster = ml_client.compute.get(cluster_name)
    print(f'Compute target named "{cluster_name}" already exists!')

except ResourceNotFoundError:
    print('Creating a new cpu compute target...')
    
    cpu_cluster = AmlCompute(
        name = cluster_name,
        size = 'STANDARD_D2_V3',
        min_instances = 0,
        max_instances = 1,
        idle_time_before_scale_down = 180,
        tier = 'Dedicated'
    )
    print(
        f"AMLCompute with name {cpu_cluster.name} will be created, with compute size {cpu_cluster.size}"
    )
    ml_client.compute.begin_create_or_update(cpu_cluster)
    print('Compute target created successfully!')

Compute target named "cpu-cluster" already exists!


In [5]:
parent_dir ='../src/python/pipeline/components'
prepare_dataset = load_component(source=os.path.join(parent_dir, 'prepare_dataset/prepare-dataset.yml'))
os.path.isfile(os.path.join(parent_dir, 'prepare_dataset/prepare-dataset.yml'))

True

In [6]:
@pipeline(name='demo_train_pipeline_with_mlflow')
def build_pipeline(raw_data):
    step_prepare_dataset = prepare_dataset(input_data=raw_data)
    
    return {
        "output_data": step_prepare_dataset.outputs.output_data
        }

In [7]:
def prepare_pipeline_job(cluster_name):
    data_asset = ml_client.data.get(name='celeba', version='initial')
    raw_data = Input(type='uri_folder', path=data_asset.path)

    pipeline_job = build_pipeline(raw_data)
    
    # set pipeline level datastore
    pipeline_job.settings.default_compute=cluster_name
    pipeline_job.settings.default_datastore="workspaceblobstore"
    pipeline_job.settings.force_rerun=False
    pipeline_job.display_name="demo_train_pipeline_with_mlflow_v2"
    
    return pipeline_job

In [8]:
prepped_job = prepare_pipeline_job(cluster_name)
ml_client.jobs.create_or_update(prepped_job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading prepare_dataset (5.47 MBs): 

Experiment,Name,Type,Status,Details Page
notebooks,sweet_oyster_qnls6kftv9,pipeline,NotStarted,Link to Azure Machine Learning studio
